In [ ]:
import requests
import re
from bs4 import BeautifulSoup
from io import BytesIO
import pandas as pd

In [5]:
def listar_e_contabilizar_links_xlsx_requests(url, filtrar_2025=True):
    """
    Analisa o HTML de uma URL usando requests e BeautifulSoup4,
    lista os links (<a> tags) que possuem a extensão .xlsx, e contabiliza quantos foram encontrados.

    Args:
        url (str): A URL da página web.

    Returns:
        tuple: Uma tupla contendo:
            - list: Uma lista de strings com os URLs dos arquivos .xlsx encontrados.
            - int: O número total de links .xlsx encontrados.
    """
    xlsx_links = []
    try:
        response = requests.get(url)
        response.raise_for_status()  # Lança uma exceção para erros HTTP
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a', href=True)
        for link in links:
            href = link['href']
            if href.lower().endswith('.xlsx'):
                # Construir o URL completo caso seja um caminho relativo
                if not href.startswith('http'):
                    from urllib.parse import urljoin
                    href = urljoin(url, href)
                    # verifica o padrão de datas 2025 se o filtro estiver ativo
                if filtrar_2025:
                    padrao = r'2025-\d{2}-\d{2}_2025-\d{2}-\d{2}\.xlsx$'
                    if re.search(padrao, href):
                        xlsx_links.append(href)
                else:   
                    xlsx_links.append(href)
        total_links = len(xlsx_links)
        return xlsx_links, total_links
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a URL: {e}")
        return [], 0
    except Exception as e:
        print(f"Ocorreu um erro ao analisar o HTML: {e}")
        return [], 0

In [ ]:
# LPC
#url_para_analisar = "https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/arquivos-lpc/2025/"
lpc_para_analisar = "https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/levantamento-de-precos-de-combustiveis-ultimas-semanas-pesquisadas"
links_encontrados_lpc, total_encontrados_lpc = listar_e_contabilizar_links_xlsx_requests(lpc_para_analisar)
if links_encontrados_lpc:
    print(f"Links .xlsx encontrados na URL '{lpc_para_analisar}':")
    for link in links_encontrados_lpc:
        print(link)
    print(f"\nTotal de {total_encontrados_lpc} links .xlsx encontrados.")
else:
    print(f"Nenhum link .xlsx encontrado na URL '{lpc_para_analisar}'.")

In [ ]:
# PDC

pdc_para_analisar = "https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/precos-de-distribuicao-de-combustiveis"
links_encontrados_pdc, total_encontrados_pdc = listar_e_contabilizar_links_xlsx_requests(pdc_para_analisar)
if links_encontrados_pdc:
    print(f"Links .xlsx encontrados na URL '{pdc_para_analisar}':")
    for link in links_encontrados_pdc:
        print(link)
    print(f"\nTotal de {total_encontrados_pdc} links .xlsx encontrados.")
else:
    print(f"Nenhum link .xlsx encontrado na URL '{pdc_para_analisar}'.")

In [ ]:
def concatenar_xlsx(urls,max_arquivos=None):
    """
    Faz o download de arquivos .xlsx a partir de uma lista de URLs e os concatena em um único arquivo.

    Args:
        urls (list): Lista de URLs dos arquivos .xlsx.
        max_arquivos (int, optional): Número máximo de arquivos a serem baixados. Se None, baixa todos.

    Returns:
        pd.DataFrame: DataFrame concatenado com os dados dos arquivos .xlsx.
    """
    import pandas as pd
    from tqdm import tqdm

    if max_arquivos is not None:
        urls = urls[:max_arquivos]

    dfs = []
    for url in tqdm(urls, desc="Baixando e concatenando arquivos"):
        try:
            response = requests.get(url)
            response.raise_for_status()
            xlsx_file = BytesIO(response.content)
            df = pd.read_excel(xlsx_file)
            dfs.append(df)
        except Exception as e:
            print(f"Erro ao baixar ou ler o arquivo {url}: {e}")

    if dfs:
        df_concatenado = pd.concat(dfs, ignore_index=True)
        return df_concatenado
    else:
        return pd.DataFrame()  # Retorna um DataFrame vazio se nenhum arquivo foi baixado